In [ ]:
import re
from glob import glob
import random
import json
import numpy as np
import json
from glob import glob
from tqdm import tqdm
import copy
import os
from config import *
from format_helper import *

# Spert: from BRAT to JSON

In [ ]:
# brat .ann to json, note level.
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
os.makedirs(f'../dataset/spert/',exist_ok=True)
for split in ['train','valid','test']:
    source_dir=f'../dataset/BRAT/{split}/*.ann'
    outfile=f'../dataset/spert/{split}_spert_note.json'
    brat2spert_note(source_dir,outfile,tokenizer)

In [ ]:
# 1. combining the assertion label to the trigger
# 2. combining the subtype label to the label

for split in ['train','valid','test']:
    # source=f'../dataset/spert/{split}_spert.json'
    # outfile=f'../dataset/spert/{split}_combined.json'
    # combine_json_attributes(source,outfile)

    source=f'../dataset/spert/{split}_spert_note.json'
    outfile=f'../dataset/spert/{split}_note_combined.json'
    combine_json_attributes(source,outfile)

In [1]:
# spert json note-level into sliding window level.
from format_helper import *
window_size=400
include_events=False
for split in ['test']:#,'train','valid','test',
    source_dir=f'../dataset/spert/{split}_note_combined.json'
    outfile=f'../dataset/spert/{split}_spert_{window_size}window.json'
    # output all chunks with all possible combinations of (start,end) sentences
    # the relation's head and tail must be either the start and end sentence
    # for the test, valid set, the head and tail entities cannot be in the same sentences, unless the chunk as only one sentence.
    max_sent=4 if split!='test' else 1000
    spertNote2spertWindow(split,source_dir,outfile,window_size,include_events,max_sent=max_sent)

# split='train'
# source_dir=f'../dataset/spert/{split}_note_combined.json'
# outfile=f'../dataset/spert/{split}_spert_{window_size}window_complete.json'
# # output all chunks with all possible combinations of (start,end) sentences
# # the relation's head and tail must be either the start and end sentence
# # for the test, valid set, the head and tail entities cannot be in the same sentences, unless the chunk as only one sentence.
# spertNote2spertWindow(split,source_dir,outfile,window_size,include_events,include_entities_side=True)

/home/velvinfu/miniconda3/envs/py310/lib/python3.10/site-packages/spacy/util.py:887: UserWarning: [W095] Model 'en_core_web_sm' (3.3.0) was trained with spaCy v3.3 and may not be 100% compatible with the current version (3.5.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
/home/velvinfu/miniconda3/envs/py310/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 115/115 [00:48<00:00,  2.40it/s]


In [ ]:
# spert json note-level into sentence_level.
from format_helper import *
for split in ['train','valid','test']:#
    source_dir=f'../dataset/spert/{split}_note_combined.json'
    outfile=f'../dataset/spert/{split}_spert_sent.json'
    spertNote2spertSent(source_dir,outfile)

In [ ]:
# countting the relation statistics to make sure consistancy
output=[key for key in relation_names]+['overall']
for split in ['train','valid','test']:
    counter={}
    for key in relation_names:
        counter[key]=0
    counter['overall']=0
    file=f'../dataset/spert/{split}_spert_400window.json'
    data=json.loads(open(file).read())
    for dic in data:
        for rel in dic['relations']:
            if rel['type'] in relation_names: # only intra-sentence relations
                counter[rel['type']]+=1
                counter['overall']+=1
    for idx, key in enumerate(counter):
        output[idx]+=f',{counter[key]}'
    
with open('../performance/relation_counts.csv','w') as f:
    f.write('\n'.join(output))   

# PL-Marker foramts

In [ ]:
from transformers import AutoTokenizer, AutoModel
from format_helper import *
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
os.makedirs(f'../dataset/plmarker/',exist_ok=True)
for split in ['valid','train','test']: #['train','valid','test']
    for type in ['sent']:
        source_file=f'../dataset/spert/{split}_spert_{type}.json'
        output_filename=f'../dataset/plmarker/{split}_{type}.jsonl'
        # setting it as none, because previous work has already counts the tokens.
        entity_types,relation_types=spert2plmarker(source_file,output_filename,tokenizer=tokenizer)

# GLM formats

In [1]:
# generative formats for events
from format_helper import *
os.makedirs(f'../dataset/GenQA/',exist_ok=True)
for split in ['train','valid','test']:
    source_file=f'../dataset/spert/{split}_spert_sent.json'
    source_dir=f'../dataset/BRAT/{split}'
    output_filename=f'../dataset/GenQA/{split}_events.json'
    spert2GLM_evemts(source_file,source_dir,output_filename,split)

/home/velvinfu/miniconda3/envs/py310/lib/python3.10/site-packages/spacy/util.py:887: UserWarning: [W095] Model 'en_core_web_sm' (3.3.0) was trained with spaCy v3.3 and may not be 100% compatible with the current version (3.5.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
/home/velvinfu/miniconda3/envs/py310/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
# generative formats for RE
from format_helper import *
os.makedirs(f'../dataset/GenQA/',exist_ok=True)
for split in ['train','valid','test']:#'train','valid','test'
    source_file=f'../dataset/spert/{split}_spert_400window.json'
    output_filename=f'../dataset/GenQA/{split}_Genre.json'
    spert2T5_generativeRE(source_file,output_filename)

/home/velvinfu/miniconda3/envs/py310/lib/python3.10/site-packages/spacy/util.py:887: UserWarning: [W095] Model 'en_core_web_sm' (3.3.0) was trained with spaCy v3.3 and may not be 100% compatible with the current version (3.5.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
/home/velvinfu/miniconda3/envs/py310/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# multi-QA formats for RE
from format_helper import *
os.makedirs(f'../dataset/GenQA/',exist_ok=True)
for split in ['train','valid','test']:#'train','valid','test'
    source_file=f'../dataset/spert/{split}_spert_400window.json'
    output_filename=f'../dataset/GenQA/{split}_re.json'
    neg_prob=0.2 if split=='train' else 1
    spert2T5QA(source_file,output_filename,neg_prob)

In [ ]:
# countting the relation statistics to make sure consistancy
output=[key for key in relation_names]+['overall']
for split in ['train','valid','test']:
    counter={}
    for key in relation_names:
        counter[key]=0
    counter['overall']=0
    file=f'../dataset/GenQA/{split}_re.json'
    data=json.loads(open(file).read())
    for dic in data:
        rel=dic['relation_source']
        if rel in relation_names: # only intra-sentence relations
            counter[rel]+=1
            counter['overall']+=1
    for idx, key in enumerate(counter):
        output[idx]+=f',{counter[key]}'
    
with open('../performance/relation_counts.csv','w') as f:
    f.write('\n'.join(output))   

# GLM predicted entitites to RE

In [ ]:
# brat .ann to json, note level.
from transformers import AutoTokenizer, AutoModel
from format_helper import *
split='test'
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
for exp in ['llama8b_events']:
    source_dir=f'../dataset/GenQA/predictions/{exp}/pred/*.ann'
    outfile=f'../dataset/GenQA/predictions/{exp}_spert_note.json'
    brat2spert_note(source_dir,outfile,tokenizer)

    source=outfile
    outfile=f'../dataset/GenQA/predictions/{exp}_note_combined.json'
    combine_json_attributes(source,outfile)

    include_events=False
    window_size=400
    source_dir=outfile
    outfile=f'../dataset/GenQA/predictions/{exp}_spert_{window_size}window.json'
    spertNote2spertWindow(split,source_dir,outfile,window_size,include_events,max_sent=1000)

    source_dir=f'../dataset/GenQA/predictions/{exp}_note_combined.json'
    outfile=f'../dataset/GenQA/predictions/{exp}_spert_sent.json'
    spertNote2spertSent(source_dir,outfile)

    source_file=f'../dataset/GenQA/predictions/{exp}_spert_400window.json'
    output_filename=f'../dataset/GenQA/predictions/{exp}_spert_GenRe.json'
    spert2T5_generativeRE(source_file,output_filename)

    source_file=f'../dataset/GenQA/predictions/{exp}_spert_400window.json'
    output_filename=f'../dataset/GenQA/predictions/{exp}_re.json'
    neg_prob=0.2 if split=='train' else 1
    spert2T5QA(source_file,output_filename,neg_prob)

# PL-Marker predicted entity to RE

In [ ]:
from transformers import AutoTokenizer, AutoModel
from format_helper import *
split='test'
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

source_dir=f'../dataset/plmarker/predictions/BRAT/*.ann'
outfile=f'../dataset/plmarker/predictions/pred_spert_note.json'
#brat2spert_note(source_dir,outfile,tokenizer)

source=outfile
outfile=f'../dataset/plmarker/predictions/pred_note_combined.json'
#combine_json_attributes(source,outfile)

include_events=False
window_size=400
source_dir=outfile
outfile=f'../dataset/plmarker/predictions/pred_spert_{window_size}window_unaligned.json'
#spertNote2spertWindow(split,source_dir,outfile,window_size,include_events,max_sent=1000, tokenizer=tokenizer)

# aligning the PL-Marker output to the original sentence. 
source_file=f'../dataset/plmarker/predictions/pred_spert_{window_size}window_unaligned.json'
outfile=f'../dataset/plmarker/predictions/pred_spert_{window_size}window.json'
reference_file='../dataset/spert/test_spert_400window.json'
#align_spert_json(source_file,reference_file,outfile)


source_file=f'../dataset/plmarker/predictions/pred_spert_{window_size}window.json'
output_filename=f'../dataset/plmarker/predictions/pred_plmarker_{window_size}window.jsonl'
# setting it as none, because previous work has already counts the tokens.
entity_types,relation_types=spert2plmarker(source_file,output_filename,tokenizer=tokenizer)

In [ ]:
source_file=f'../dataset/plmarker/predictions/pred_spert_{window_size}window_unaligned.json'
outfile=f'../dataset/plmarker/predictions/pred_spert_{window_size}window.json'
reference_file='../dataset/spert/test_spert_400window.json'

def align_spert_json(source_file,reference_file,outfile):
    pred=json.loads(open(source_file).read())
    source=json.loads(open(reference_file).read())

    for s in tqdm(source):
        s['entities']=[]
        s['relations']=[]
        #s['tokens']=[t['text'] for t in s['tokens']]
        for p in pred:
            if p['id']==s['id']:
                s['entities']=copy.deepcopy(p['entities'])
                break

    with open(outfile,'w') as f:
        json.dump(source,f,indent=4)
    return

align_spert_json(source_file,reference_file,outfile)

In [ ]:
outfile=f'../dataset/plmarker/predictions/pred_spert_{window_size}window.json'
source=json.loads(open(outfile).read())

for s in tqdm(source):
    s['tokens']=[t['text'] for t in s['tokens']]


with open(outfile,'w') as f:
    json.dump(source,f,indent=4)